# Ray Tasks in Practice: Building Distributed Applications

© 2025, Anyscale. All Rights Reserved

This notebook provides a step-by-step introduction to Ray Tasks, the fundamental building block of Ray that enables distributed computing.

<div class="alert alert-block alert-info">

<b> Here is the roadmap for this notebook </b>

<ol>
  <li>Overview and setup</li>
  <li>Simple task submission (creating, executing, and getting results)</li>
  <li>Task options and configuration</li>
  <li>Object store and memory model</li>
  <li>Chaining tasks and passing data</li>
  <li>Error handling and task retries</li>
  <li>Task runtime environments</li>
  <li>Resource allocation and management</li>
  <li>Pipeline data processing and waiting for results</li>
  <li>Ray generators</li>
</ol>
</div>

## Imports


In [ ]:
import math
import os
import random
import time

import numpy as np
import pandas as pd
import ray
import requests
import ray.runtime_context
from ray import tune
from ray.util.scheduling_strategies import NodeAffinitySchedulingStrategy

## 1. Overview and setup

### 1.1. Ray Core at a glance

- **Scales your code** across many CPU cores, machines, and accelerators.  
- **Schedules arbitrary task graphs** thanks to its distributed scheduler.
- **Hides distributed-system overhead** with built-ins for  
  - fast data serialization and transfer,  
  - smart task placement, 
  - distributed memory & reference counting.

Ray's higher-level libraries build on Ray Core to offer ready-made APIs for common workloads.

### 1.2. When to use Ray Tasks

Ray Tasks are ideal for:
- **Parallelizing computationally expensive functions** across multiple cores or machines
- **Processing large datasets** by distributing work across workers
- **Building complex task dependency graphs** (DAGs) for data pipelines
- **Scaling existing Python code** with minimal changes

**When NOT to use Ray Tasks:**
- Functions that execute in < 1ms (overhead not worth it)
- Very fine-grained parallelism (use NumPy/Pandas instead)
- When you need mutable shared state (use Ray Actors instead)

### 1.3. Ray cluster architecture

Before diving into tasks, let's understand the key components of a Ray cluster.

<img src="https://docs.ray.io/en/latest/_images/ray-cluster.svg" width="800">

A Ray cluster consists of:
- One or more **worker nodes**, where each worker node consists of the following processes:
    - **worker processes** responsible for task submission and execution.
    - A **raylet** responsible for:
      - resource management and task placement.
      - shared memory management through an object store 
- One of the worker nodes is designated a **head node** and is responsible for running 
  - A **global control service** responsible for keeping track of the **cluster-level state** that is not supposed to change too frequently.
  - An **autoscaler** service responsible for adding and removing worker nodes by integrating with different infrastructure providers (e.g. AWS, GCP, ...) to match the resource requirements of the cluster.


### 1.4. Initializing Ray

Before using Ray, you must initialize it. Here are common initialization patterns:


In [ ]:
# Connect to existing Ray cluster or create a new one in local development (single machine)
ray.init(ignore_reinit_error=True)

## 2. Simple task submission (creating, executing, and getting results)

### 2.1. Creating remote functions

The first step in using Ray is to create remote functions. A remote function is a regular Python function that can be executed on any process in your cluster.

Given a simple Python function:


In [ ]:
def add(a, b):
    return a + b

add

Decorate the function with `@ray.remote` to turn it into a remote function.


In [ ]:
@ray.remote
def remote_add(a, b):
    return a + b

remote_add

### 2.2. Executing remote functions (asynchronous by default)

Native python functions are invoked by calling them:


In [ ]:
add(1, 2)  # Returns 3 immediately

Remote ray functions are executed as tasks by calling them with `.remote()` suffix:


In [ ]:
remote_add.remote(1, 2)  # Returns ObjectRef immediately, computation happens async

<div class="alert alert-info">
  <strong><a href="https://docs.ray.io/en/latest/ray-core/key-concepts.html#tasks" target="_blank">Tasks</a></strong> is a remote, stateless Python function invocation.
</div>

Here is what happens when you call `{remote_function}.remote`:
1. Ray schedules the function execution as a task in a separate process in the cluster
2. Ray returns an `ObjectRef` (a reference to the future result) to you **immediately** 
3. The cluster executes the actual computation in the background


In [ ]:
ref = remote_add.remote(1, 2)
ref

**Think of `ObjectRef` as a future/promise**: it's a placeholder for a value that will be computed later.

Here is a map of how Python code is translated into Ray tasks:

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/ray-core/python_to_ray_task_map_v2.png" alt="Python to Ray Task Map" width="800">

### 2.3. Getting results

If we want to wait (block) and retrieve the corresponding object, we can use `ray.get`:


In [ ]:
ray.get(ref)

`ray.get()` works with single ObjectRefs or lists:


In [ ]:
# Single result
result = ray.get(ref)

# Multiple results
refs = [remote_add.remote(i, i) for i in range(5)]
results = ray.get(refs)  # Wait for all to complete
print(results)  # [0, 2, 4, 6, 8]

### 2.4. Putting it all together

Here are the three steps:
1. Create the remote function
2. Execute it remotely (non-blocking)
3. Get the result when needed (blocking)

<div class="alert alert-block alert-info">
    
__Activity: define and invoke a Ray task__

Define a remote function `sqrt_add` that accepts two arguments and performs the following steps:
1. computes the square-root of the first
2. adds the second
3. returns the result

Execute it with 2 different sets of parameters and collect the results


In [ ]:
# Hint: define the below as a remote function
def sqrt_add(a, b):
    ... 

# Hint: invoke it as a remote task and collect the results

</div>


In [ ]:
# Write your solution here

<div class="alert alert-block alert-info">

<details>

<summary> Click to see solution </summary>

In [ ]:
@ray.remote
def sqrt_add(a, b):
    return math.sqrt(a) + b

ray.get([sqrt_add.remote(2, 3), sqrt_add.remote(5, 4)])

</details>

### 2.5. Understanding asynchronous execution

The key difference between regular Python and Ray is that `.remote()` **does not block**:


In [ ]:
def slow_function(x):
    time.sleep(3)
    return x * x

# Sequential Python (blocks for each call)
start = time.time()
results = [slow_function(i) for i in range(4)]  # Would take 12 seconds!
print(f"Sequential: {time.time() - start:.2f}s")

slow_function = ray.remote(slow_function)

# Distributed Ray (non-blocking)
start = time.time()
refs = [slow_function.remote(i) for i in range(4)]  # Returns immediately!
print(f"Task submission: {time.time() - start:.2f}s")  # < 0.01s

# Now wait for results (blocks until all complete)
results = ray.get(refs)
print(f"Total with parallelism: {time.time() - start:.2f}s")  # ~3s

</details>

</div>

### 2.6. What can be passed to Ray tasks? (Serialization)

Ray uses **cloudpickle** to serialize function arguments and return values. Most Python objects work, but there are limitations:

**✅ Can serialize:**
- Basic types: int, float, str, bool, None
- Collections: list, dict, tuple, set
- NumPy arrays, Pandas DataFrames
- Most custom classes
- ObjectRefs (passed efficiently by reference)

**❌ Cannot serialize:**
- File handles (`open()` objects)
- Network sockets
- Threading locks

**Example of serialization issues:**


In [ ]:
# ❌ BAD: File handle won't serialize
file = open("/tmp/data.txt", "w")

@ray.remote
def read_file(f):
    return f.read()

# ref = read_file.remote(file)  # Will fail with a PicklingError

<div class="alert alert-warning">
<b>💡 Troubleshooting:</b> If you see <code>pickle.PicklingError</code> or <code>TypeError: cannot pickle</code>, check if you're passing non-serializable objects to your task.
</div>

### 2.7 Task submission sequence

Here is the sequence of events when you submit a Ray task:

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/ray-core/task-submission_old.gif" alt="Task Submission Sequence" width="800">

### 2.8 Task submission under the hood

When a task is submitted, here is how resource fulfillment works

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/ray-core/normal-task-resource-fullfilment.svg" width="700" alt="Resource fulfillment and execution of `double(2)` in a Ray cluster.">

Tthe caller must choose **which node (raylet)** should schedule it.

#### 1️⃣ Choosing the Preferred Raylet
| **Rule**          | **When Used**                    | **How It Works**                                                                                    |
| ----------------- | -------------------------------- | --------------------------------------------------------------------------------------------------- |
| **Data locality** | Task has large input objects.    | Pick node holding the most object bytes locally (from the object directory, may be slightly stale). |
| **Node affinity** | Task specifies a target node.    | Use the node from `NodeAffinitySchedulingStrategy`.                                                 |
| **Default**       | No data or affinity preferences. | Use the **local raylet**.                                                                           |


#### 2️⃣ Request → Lease → Worker
- Caller sends a **resource request** to the preferred raylet.  
- If granted, the raylet **leases a local worker** and returns its address.  
- The **lease stays active** while both caller and worker are alive.  
- Idle or unused leases are returned after a short timeout (~hundreds of ms).


#### 3️⃣ Task Execution on the Leased Worker
The caller can schedule **multiple compatible tasks** on the same worker without re-contacting the scheduler.

Compatibility means matching:
- **Resource shape**, e.g. `{"CPU": 1}`
- **Shared-memory arguments** (large objects must be local; small ones are inlined)
- **Runtime environment**


#### 4️⃣ Optimization Insight
Worker leases act as a *cache* for scheduling decisions — similar tasks can reuse the same worker for lower latency and higher throughput.

Note also that the caller can hold multiple worker leases to increase parallelism. 

<!-- TODO - perhaps add the example debug logs here -->


## 3. Task options and configuration

You can dynamically configure tasks using the `.options()` method without redefining the function. This is useful for adjusting resources, retries, or other settings per task invocation.

### 3.1. Basic usage of .options()


In [ ]:
@ray.remote
def flexible_task(x):
    return x * 2

# Use default configuration (1 CPU)
ref1 = flexible_task.remote(5)

# Override to use 2 CPUs for this specific invocation
ref2 = flexible_task.options(num_cpus=2).remote(10)

### 3.2. Common options

**Resource options:**
- `num_cpus`: Number of CPUs (can be fractional, e.g., 0.5)
- `num_gpus`: Number of GPUs (can be fractional)
- `memory`: Memory in bytes
- `resources`: Dict of custom resources

**Fault tolerance options:**
- `max_retries`: Max number of retries (default: 3 for system errors)
- `retry_exceptions`: List of exception types to retry on

**Execution options:**
- `runtime_env`: Dict specifying runtime environment
- `scheduling_strategy`: Control task placement
- `name`: Name for debugging/monitoring

### 3.3. Scheduling strategies


In [ ]:
# Default: Ray decides based on data locality and load
flexible_task.remote(4)

# SPREAD: Distribute tasks across nodes
flexible_task.options(scheduling_strategy="SPREAD").remote(5)

# Node affinity: Run on specific node
strategy = NodeAffinitySchedulingStrategy(
    node_id=ray.get_runtime_context().get_node_id(),
    soft=True  # soft=True allows fallback if node unavailable
)
flexible_task.options(scheduling_strategy=strategy).remote(6)

## 4. Object store and memory model

Each worker node has its own object store, and collectively, these form a shared object store across the cluster.

Remote objects are immutable. That is, their values cannot be changed after creation. This allows remote objects to be replicated in multiple object stores without needing to synchronize the copies.

| <img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/ray-core/ray-cluster.png" width="700px" loading="lazy"> |
| :---------------------------------------------------------------------------------------------------------------------------- |
| A Ray cluster with a head node and two worker nodes. Highlighted in orange is distributed object store.                       |

<div class="alert alert-info">
  <strong><a href="https://docs.ray.io/en/latest/ray-core/key-concepts.html#objects" target="_blank">Object</a></strong> - tasks and actors create and work with remote objects, which can be stored anywhere in a cluster. These objects are accessed using <strong>ObjectRef</strong> and are cached in a distributed shared-memory <strong>object store</strong>.
</div>

### 4.1. Ray memory model

Ray manages memory in several ways to efficiently handle distributed tasks:

1. **Heap memory**:
   - Used by workers to execute tasks and actors.
   - Used to store small objects (less than 100KB) and Ray metadata.
   - High memory pressure can cause Ray to terminate some tasks to free up resources.

2. **Shared memory (Object Store)**:
   - Serves as the medium for passing data between tasks.
   - Large objects (greater than 100KB) are stored in a shared memory space, using up to 30% of a node's memory.
   - If more space is needed, objects can be spilled to disk or stored on disk in a slower-access format.

#### When objects are stored in the object store

**Understanding the 100KB threshold:**


In [ ]:
@ray.remote
def small_return():
    return 42  # < 100KB: Returned inline (not stored in object store)

@ray.remote
def large_return():
    return np.random.rand(1000, 1000)  # 8MB: Implicitly stored in object store

# Explicitly store in object store
large_data = np.random.rand(10000, 10000)  # 800MB
ref = ray.put(large_data)  # Stored in object store

**Flow diagram:**
```
Task returns value
     │
     ├─→ < 100KB? ─→ Yes ─→ Send inline to caller (fast)
     │
     └─→ No ─→ Store in object store (shared memory)
                    │
                    └─→ Caller gets ObjectRef
```

#### Design tradeoffs: Choosing where to store objects

The following table outlines the tradeoffs between storing objects in the heap memory (in-process store) versus the shared memory (distributed object store):

| **Design Aspect**           | **In-Process Store**                      | **Distributed Object Store**                                |
| --------------------------- | ----------------------------------------- | ----------------------------------------------------------- |
| **Resolution Time**         | Fast (direct memory copy)                 | Slower (requires RPCs)                                      |
| **Memory Footprint**        | Higher (multiple copies across processes) | Lower (shared memory reduces duplication)                   |
| **Throughput**              | Limited by owner process                  | Scales with number of nodes                                 |
| **Data Sharing**            | Copies needed for multiple processes      | Shared memory allows multiple processes to access           |
| **Object Size Constraints** | Limited by machine's memory capacity      | Can reference objects larger than a single machine's memory |

This setup allows Ray to optimize performance and resource usage in distributed applications.

Here is a simplified diagram of the Ray memory model:

<img src="https://docs.ray.io/en/latest/_images/memory.svg" width="600">

#### Object store capacity and OOM errors

The object store has limited capacity (default: 30% of RAM). When full:

1. Ray will spill objects to disk (slower access)
2. If disk is also full, you'll get `ObjectStoreFullError`

<div class="alert alert-warning">
<b>💡 Troubleshooting:</b> If you see <code>ObjectStoreFullError</code>, you're creating objects faster than they're being consumed. 

Solutions:
<ul>
  <li>Process results in batches</li>
  <li>Use Ray Generators for streaming</li>
  <li>Increase object store size with <code>ray.init(object_store_memory=...)</code></li>
</ul>
</div>

### 4.2. Example: Producer-consumer pattern with numpy arrays

This example demonstrates how Ray transfers data in the distributed object store. The `producer_task` creates a 4 GiB numpy array, and the `consumer_task` accesses it with zero-copy deserialization when on the same node:


In [ ]:
@ray.remote
def producer_task(size_mb: int = 4 * 1024) -> np.ndarray:
    array = np.random.rand((1024**2 * size_mb // 8)).astype(np.float64)
    return array


@ray.remote
def consumer_task(array: np.ndarray) -> None:
    assert isinstance(array, np.ndarray)
    assert not array.flags.owndata  # Confirms zero-copy

arr_ref = producer_task.remote()  # Produce a 4 GiB array
output_ref = consumer_task.remote(arr_ref)  # Pass ObjectRef to consumer

**What happens under the hood:**

1. **Producer task** creates the array in heap memory, then Ray stores it in the shared object store (large objects > 100KB)
2. **Consumer task** receives the `ObjectRef` and directly accesses the array from shared memory with zero-copy deserialization (if on same node)
3. If tasks run on different nodes, Ray copies the array across the network only once

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/ray-data-deep-dive/producer-consumer-object-store-v2.png" width="600">

To see memory usage in action, run this inspection script:


In [ ]:
!python scripts/memory_inspection.py

#### Deep dive: How zero-copy deserialization works

Ray uses **cloudpickle** and **pickle 5** for zero-copy deserialization. When workers on the same node access an object, Ray returns a **view** of shared memory instead of copying data to the process heap.

**Zero-copy protocol requirements:**

Zero-copy deserialization requires that the object supports the **Pickle 5 out-of-band buffer protocol**. This protocol allows the serializer to separate the object's metadata from its raw buffer data, enabling shared memory access.

**When zero-copy is used:**
- Large objects shared by workers on the same node
- Write-once, read-many-times semantics
- Contiguous numpy arrays (currently the main supported type)

**How Ray transfers code and data:**

1. **Code transfer (functions)**:
   - Functions are pickled and stored in the Global Control Store (GCS)
   - Subsequent calls fetch the function definition from a cache

2. **Data transfer (arguments/return values)**:
   - **Small objects (< 100 KB)**: Pickled and transferred inline with the task metadata
   - **Large objects (> 100 KB)**: Pickled, stored in shared memory (object store), and only the `ObjectRef` is transferred

**Performance considerations:**
  
- **Zero-copy limitations**:
  - Currently works only for **contiguous numpy arrays**
  - Cannot modify shared objects directly (requires copy-on-write)
  - Does not work for PyTorch tensors or other array types
  - **Workaround**: Convert objects to contiguous numpy arrays before passing to tasks:
    ```python
    # Convert PyTorch tensor to numpy
    numpy_array = tensor.cpu().numpy()
    
    # Ensure contiguous layout
    contiguous_array = np.ascontiguousarray(numpy_array)
    
    ref = ray.put(contiguous_array)
    ```

- **Cloudpickle overhead**: Cloudpickle has fixed serialization overhead. For small objects where performance matters, consider using raw `bytes` and performing your own serialization to bypass cloudpickle.

To read more about object serialization in Ray, see [this documentation page here](https://docs.ray.io/en/latest/ray-core/objects/serialization.html).

**Mutability semantics:**

Objects in the object store are **immutable once sealed**. This design choice enables safe sharing across multiple processes without locks, but means in-place updates won't propagate to other workers accessing the same `ObjectRef`.

### 4.3. Usecase: Hyper-parameter tuning

**The Problem**: When running hyperparameter tuning or experimentation, you often need to use the same dataset across dozens or hundreds of trials. If you pass the dataset by value to each training function, Ray will serialize it repeatedly, wasting memory and time.

**The Solution**: Store the dataset once in the object store using `ray.put()`, then pass only the lightweight `ObjectRef` to each trial. All workers can access the same data without duplication.

#### Real-world scenario: Grid search with shared data

Imagine running 20 experiments on a 100MB training dataset:

In [ ]:
# Simulate a 100MB training dataset
df = pd.DataFrame(np.random.rand(100 * 1024 ** 2 // 8))

@ray.remote
def train_model(data, learning_rate, batch_size):
    # Simulate model training
    result = data.mean().sum() * learning_rate / batch_size
    time.sleep(20)
    return {"lr": learning_rate, "batch_size": batch_size, "score": result}

# Grid search: 20 different hyperparameter combinations
hyperparameters = [
    {"lr": lr, "batch_size": bs}
    for lr in [0.001, 0.01, 0.1, 0.5]
    for bs in [32, 64, 128, 256, 512]
]

Here is an efficient way to run the experiments by passing the dataset by reference:

In [ ]:
# ✅ Memory Efficient: Pass ObjectRef (100 MB total memory)
# Ray serializes once, all workers share the same data
df_ref = ray.put(df)
[
    train_model.remote(df_ref, hp["lr"], hp["batch_size"]) 
    for hp in hyperparameters
]
print("Pass by reference: ~100 MiB memory used")

Let's inspect the object store, we should only see the same 100MiB object being used across tasks

In [ ]:
!ray list objects

Here is the inefficient way by passing the dataset by value:

In [ ]:
# ❌ Memory inefficient: Pass dataframe by value (2 GB total memory!)
# Ray serializes 100MB × 20 times = 2 GB of redundant data
[
    train_model.remote(df, hp["lr"], hp["batch_size"]) 
    for hp in hyperparameters
]
print("Pass by value: ~2GiB memory used")

Let's inspect the object store, we should now see different 100MiB objects being used across tasks

In [ ]:
!ray list objects 

**Performance comparison:**
- **Pass by value**: 2 GB memory used (20× serialization overhead)
- **Pass by reference**: 100 MB memory used (1× serialization)

**Rule of thumb**: Pass by value only for small literals (< 100 KiB); otherwise, pass by reference.

#### How Ray Tune leverages this pattern

Ray Tune uses `tune.with_parameters()` to automatically pass large constant objects via the object store:

In [ ]:
def trainable(config, data):
    # Each trial receives a reference to the shared data
    model = train(data, lr=config["lr"], epochs=config["epochs"])
    return {"accuracy": model.eval()}

# Tune automatically stores train_data in the object store
tuner = tune.Tuner(
    tune.with_parameters(trainable, data=df),  # Passed by reference
    param_space={"lr": tune.grid_search([0.001, 0.01, 0.1]), "epochs": tune.choice([10, 20, 50])},
)

Without `tune.with_parameters()`, each trial would receive a separate copy of `train_data`, multiplying memory usage by the number of concurrent trials.

### 4.4. Distributed ownership and fate-sharing

Ray uses a **distributed ownership model** to manage objects efficiently across the cluster. Understanding this concept is crucial for building robust distributed applications.

#### How distributed ownership works

In Ray, the process that creates or submits a task becomes the **owner** of the task's result. The owner maintains critical metadata about the object, including:
- Object location(s) in the cluster
- Reference counts
- Object size and other properties

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/distributed_ownership_overview_v4.svg" width="800px">

**Benefits of distributed ownership:**
- **Lower latency**: No need to communicate all ownership information back to a central node
- **Better scalability**: No single bottleneck as every worker maintains its own ownership information

Here is a diagram that explains how distributed ownership works in a Ray cluster:

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/ray-core/distributed-ownership.png" width="800">

Below is some code based on the above diagram to illustrate distributed ownership:

In [ ]:
@ray.remote
def b():
    size_mib = 19
    return np.ones(1024 ** 2 // 8 *  size_mib)

@ray.remote
def a(dep):
    z = b.remote()

    ip = ray.util.get_node_ip_address()
    print(f"{ip=}")

    time.sleep(20)
    return dep.sum() / ray.get(z).sum() 

size_mib = 33
arr = np.ones(1024 ** 2 // 8 *  size_mib)
x = ray.put(arr)
y = a.remote(x)

We can verify that the the 19 MB array is owned by the worker that submitted task `b` - i.e. the worker executing task `a` 

In [ ]:
!ray list objects --filter TASK_STATUS!=NIL --filter TYPE=WORKER  # TASK_STATUS = NIL is non-owners given Task only the owner tracks task status

In [ ]:
ray.get(y), 33 / 19

#### The fate-sharing limitation

The main tradeoff of distributed ownership is **fate-sharing**: objects are tied to the lifetime of their owner process.

**What this means:**
- Even if an object is stored in the object store on a different node, if the owner process dies, the object becomes unreachable
- The owner maintains critical metadata (locations, reference counts) that other processes need to access the object
- When the owner fails, this metadata is lost, making the object inaccessible even if copies exist elsewhere

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/distributed_ownership_fate_share_with_owner_v4.svg" width="900px">

#### Example: Demonstrating fate-sharing

This example creates two actors: an **Owner** that creates an object reference, and a **Borrower** that tries to access it. We'll see what happens when the Owner is terminated:


In [ ]:
@ray.remote
def f(data):
    return data

@ray.remote
class Owner:
    def __init__(self):
        self.ref = None

    def set_object_ref(self, data):
        self.ref = f.remote(data)
        return self.ref
    
    def is_alive(self): 
        return True

@ray.remote
class Borrower:
    def get_object(self, ref):
        return ray.get(ref)

owner = Owner.remote()
borrower = Borrower.remote()
assert ray.get(owner.is_alive.remote())

object_ref = owner.set_object_ref.remote(data="test1")
data = ray.get(borrower.get_object.remote(object_ref))
assert data == "test1"
print(f"✓ Successfully retrieved data while Owner is alive: {data}")

ray.kill(owner)
time.sleep(2)

try:
    ray.get(borrower.get_object.remote(object_ref))
    print("✗ Unexpected: Should have failed!")
except Exception as e:
    print("✓ Failed as expected after owner termination:")
    print(e)

**What happens:**
1. While the Owner is alive, the Borrower can successfully retrieve the object using the `ObjectRef`
2. After the Owner is killed, the Borrower still has the `ObjectRef`, but attempting to access the object fails
3. Even though the object data may still exist in the object store, the ownership metadata is lost

**Key takeaway:** In Ray's distributed ownership model, object lifetime is tied to the owner's lifetime. When building fault-tolerant applications:
- Keep important owners alive (e.g., use long-running actors or the driver process)
- Consider checkpointing critical data outside Ray's object store for durability

### 4.5 Lineage Reconstruction

If instead, the owner is still alive, but the object is lost (e.g., due to node failure), Ray can reconstruct the object by either

1. Finding any secondary copies in the object store (if they exist) and returning one of those
2. Re-executing the task or task chain that created it. This is known as **lineage reconstruction**.

See the below test inspired from the [Ray test suite](https://github.com/ray-project/ray/blob/a04cb06bb1a2c09e93b882b611492d62b8d1837a/python/ray/tests/test_reconstruction.py#L126) for an example of lineage reconstruction:

```python
@pytest.mark.parametrize("reconstruction_enabled", [False, True])
def test_basic_reconstruction(config, ray_start_cluster, reconstruction_enabled):
    cluster = ray_start_cluster
    # Head node with no resources.
    cluster.add_node(
        num_cpus=0,
        _system_config=config,
        enable_object_reconstruction=reconstruction_enabled,
    )
    ray.init(address=cluster.address)
    # Node to place the initial object.
    node_to_kill = cluster.add_node(
        num_cpus=1, resources={"node1": 1}, object_store_memory=10**8
    )
    cluster.wait_for_nodes()

    @ray.remote(max_retries=1 if reconstruction_enabled else 0)
    def create_large_object():
        return np.zeros(10**7, dtype=np.uint8)

    @ray.remote
    def process_large_object(x):
        return

    # Create a large object on node1
    obj = create_large_object.options(resources={"node1": 1}).remote()
    # Create a dependent task that will use the large object
    ray.get(process_large_object.options(resources={"node1": 1}).remote(obj))
    # Remove the node that has the large object
    cluster.remove_node(node_to_kill, allow_graceful=False)
    # Add a new node with the same resource config
    node_to_kill = cluster.add_node(
        num_cpus=1, resources={"node1": 1}, object_store_memory=10**8
    )

    if reconstruction_enabled:
        # If the large object is lost and reconstruction is enabled, the following call will succeed
        # i.e create_large_object will get called again and the object will be reconstructed
        # then process_large_object can run successfully
        ray.get(process_large_object.remote(obj))
    else:
        # Both the dependent task and the object will be lost
        with pytest.raises(ray.exceptions.RayTaskError):
            ray.get(process_large_object.remote(obj))
        with pytest.raises(ray.exceptions.ObjectLostError):
            ray.get(obj)

    # Losing the object a second time will cause reconstruction to fail because
    # we have reached the max task retries.
    cluster.remove_node(node_to_kill, allow_graceful=False)
    cluster.add_node(num_cpus=1, resources={"node1": 1}, object_store_memory=10**8)

    if reconstruction_enabled:
        with pytest.raises(
            ray.exceptions.ObjectReconstructionFailedMaxAttemptsExceededError
        ):
            ray.get(obj)
    else:
        with pytest.raises(ray.exceptions.ObjectLostError):
            ray.get(obj)
```

### 4.6 ObjectRef lifecycle and garbage collection

Objects in the object store are automatically garbage collected when teir distributed reference count drops to zero. This happens when all `ObjectRef`s pointing to the object are deleted or go out of scope.

**Example:**


In [ ]:
@ray.remote(num_returns=2)
def create_object():
    task_id = ray.runtime_context.get_runtime_context().get_task_id()
    return np.random.rand(1024 ** 2 // 8 * 20), task_id

# Object created and stored
ref1, ref2 = create_object.remote()

# Object still in memory
result = ray.get(ref1)
task_id = ray.get(ref2)

Let's inspect the returned objects in the store - note in this case we leverage the object id specification to find the object in the store

In [ ]:
!ray list objects --filter TASK_STATUS!=NIL --filter TYPE=DRIVER --filter OBJECT_ID={task_id}01000000

In [ ]:
# Release reference (allows GC)
del ref1
del ref2

In [ ]:
!ray list objects --filter TASK_STATUS!=NIL --filter TYPE=DRIVER --filter OBJECT_ID={task_id}02000000

<div class="alert alert-info">
<b>⚡ Performance Tip:</b> For long-running applications, explicitly delete ObjectRefs you no longer need to free up object store memory.
</div>

## 5. Chaining tasks and passing data

Let's say we now want to execute a graph of two tasks:
1. Square a value using `expensive_square`
2. Add 1 to the `expensive_square` result, by using `remote_add`

In [ ]:
@ray.remote
def expensive_square(x):
    time.sleep(1)
    return x**2

This can be achieved without fetching an intermediate result.

**❌ Anti-pattern:**

In [ ]:
# 1st task
square_ref = expensive_square.remote(2)
square_value = ray.get(square_ref)  # wait to get the value

# 2nd task
sum_ref = remote_add.remote(1, square_value)  # pass value from 1st task
sum_value = ray.get(sum_ref)

**✅ Better:** Chain the tasks by passing the `ObjectRef` directly to the second task:


In [ ]:
square_ref = expensive_square.remote(2)
sum_ref = remote_add.remote(1, square_ref)  # Pass ObjectRef, not value!
sum_value = ray.get(sum_ref)  # Wait only at the end

**Why this is better:**
- No unnecessary data transfer (ObjectRef is just an ID)
- Ray automatically handles dependencies
- Second task waits for first task to complete
- More efficient scheduling

### 5.1. Common task graph patterns

Ray excels at executing complex directed acyclic graphs (DAGs) of tasks:

#### Pattern 1: Linear chain


In [ ]:
@ray.remote
def step1(data):
    return process_a(data)

@ray.remote
def step2(data):
    return process_b(data)

@ray.remote
def step3(data):
    return process_c(data)

# Chain tasks
# ref1 = step1.remote(input_data)
# ref2 = step2.remote(ref1)
# ref3 = step3.remote(ref2)
# final_result = ray.get(ref3)

#### Pattern 2: Fan-out / Fan-in (MapReduce)


In [ ]:
@ray.remote
def map_task(chunk):
    return process_chunk(chunk)

@ray.remote
def reduce_task(results):
    return aggregate(results)

# Map phase (fan-out)
# map_refs = [map_task.remote(chunk) for chunk in data_chunks]

# Reduce phase (fan-in)
# final_result = ray.get(reduce_task.remote(map_refs))

#### Pattern 3: Tree reduction


In [ ]:
@ray.remote
def pairwise_sum(a, b):
    return a + b


refs = [ray.put(i) for i in range(16)]  # Initial values

# Tree reduction (depth = log2(16) = 4)
while len(refs) > 1:
    refs = [pairwise_sum.remote(refs[i], refs[i + 1]) for i in range(0, len(refs), 2)]

result = ray.get(refs[0])

### 5.2. Nested tasks

Tasks can submit other tasks, enabling dynamic workflows:

In [ ]:
@ray.remote
def main():
    square_ref_1 = expensive_square.remote(1)
    square_ref_2 = expensive_square.remote(2)
    add_ref = remote_add.remote(square_ref_1, square_ref_2)
    return ray.get(add_ref)

ray.get(main.remote())

**Avoiding deadlocks:** Ray automatically yields CPU resources when blocked on `ray.get()`, preventing deadlocks when nested tasks need the same resources.

In [ ]:
# Cluster has 2 CPUs total

@ray.remote(num_cpus=2)
def outer_task():
    inner_refs = [inner_task.remote() for _ in range(10)]
    return ray.get(inner_refs)  # Ray yields the 2 CPUs while waiting

@ray.remote(num_cpus=1)
def inner_task():
    return 

ray.get(outer_task.remote())  # Works! No deadlock

<div class="alert alert-info">
Read more about <strong><a href="https://docs.ray.io/en/latest/ray-core/tasks/nested-tasks.html#yielding-resources-while-blocked" target="_blank">yielding resources while blocked</a></strong>.
</div>

## 6. Error handling and task retries

### 6.1. Understanding exception types

Let's consider two types of exceptions:
1. **System errors**: Worker node dies, out of memory, network issues
2. **Application-level errors**: Python exceptions in your code (ValueError, TypeError, etc.)

Ray will automatically **retry a task up to 3 times** if it fails due to a system error (e.g., a worker node dies).

### 6.2. Handling application exceptions

Below task won't be retried by default because it's an application failure:


In [ ]:
@ray.remote
def incorrect_square(x: int, prob: float) -> int:
    if random.random() < prob:
        raise ValueError("Random failure")
    return x**2

In [ ]:
try:
    ray.get([incorrect_square.remote(x=4, prob=0.5) for _ in range(10)])
except ray.exceptions.RayTaskError as e:
    print(f"Task failed with: {e}")
    print(f"Original exception: {e.cause}")  # Access underlying exception

**Exception propagation:**
- Exceptions in tasks are wrapped in `RayTaskError`
- The original exception is available via `.cause` attribute
- `ray.get()` will raise the exception
- ObjectRefs remain valid, but getting them raises the exception

### 6.3. Configuring retries

Ray lets you specify how to handle retries when an exception is encountered:


In [ ]:
@ray.remote(retry_exceptions=[ValueError])
def correct_square(x: int, prob: float) -> int:
    if random.random() < prob:
        raise ValueError("Random failure")
    return x**2

Note we did not have to re-define the remote function, instead we can create an updated version using `.options`:


In [ ]:
correct_square_mod = correct_square.options(
    retry_exceptions=[ValueError],
    max_retries=10,
)

Let's try it out:


In [ ]:
try:
    outputs = ray.get([correct_square_mod.remote(x=4, prob=0.5) for _ in range(10)])
    print(f"Success! Results: {outputs}")
except ray.exceptions.RayTaskError:
    print("At least one of the tasks failed after all retries")

### 6.4. Idempotency: Critical for reliable retries

**⚠️ WARNING:** Only retry tasks that are **idempotent** (can be safely run multiple times).

**❌ Non-idempotent (dangerous to retry):**


In [ ]:
@ray.remote(retry_exceptions=[ValueError])
def append_to_file(data):
    with open("/tmp/data.txt", "a") as f:
        f.write(data)  # Will duplicate data on retry!
    if random.random() < 0.5:
        raise ValueError("Simulated failure")
    return "done"

**If this task fails and retries:**
1. First attempt: Writes "hello" → fails
2. Retry: Writes "hello" again → file now has "hellohello"

**✅ Idempotent (safe to retry):**


In [ ]:
@ray.remote(retry_exceptions=[ValueError])
def write_to_file_safe(data, unique_id):
    filename = f"data_{unique_id}.txt"
    with open(filename, "w") as f:  # Overwrites on retry
        f.write(data)
    if random.random() < 0.5:
        raise ValueError("Simulated failure")
    return "done"

**Other idempotent operations:**
- Reading from database
- GET requests (not POST/PUT/DELETE)
- Mathematical computations
- Overwriting files (not appending)

**Non-idempotent operations to avoid retrying:**
- Appending to files/databases
- Sending emails/notifications
- Charging credit cards
- Incrementing counters

### 6.5. Task timeouts and cancellation

Sometimes you want to set a maximum execution time or cancel tasks:

#### Setting timeouts with ray.get()


In [ ]:
@ray.remote
def slow_task():
    time.sleep(100)
    return "done"

ref = slow_task.remote()

try:
    result = ray.get(ref, timeout=5)  # Wait max 5 seconds
except ray.exceptions.GetTimeoutError:
    print("Task took too long!")
    ray.cancel(ref)  # Cancel the task

#### Cancelling tasks


In [ ]:
@ray.remote
def long_running_task(duration):
    time.sleep(duration)
    return "completed"

refs = [long_running_task.remote(10) for _ in range(5)]

# Cancel all tasks
for ref in refs:
    ray.cancel(ref)

# Check if cancelled
try:
    ray.get(refs[0])
except ray.exceptions.TaskCancelledError:
    print("Task was cancelled")

**Important notes about cancellation:**
- Cancellation is best-effort, not guaranteed
- Task might complete before cancellation takes effect
- Dependent tasks are also cancelled
- Use for cleanup, not critical functionality

#### Using ray.wait() with timeouts


In [ ]:
refs = [slow_task.remote() for _ in range(10)]

# Wait for at least 3 tasks to complete, timeout after 30 seconds
ready, not_ready = ray.wait(refs, num_returns=3, timeout=30, fetch_local=False)

print(f"Completed: {len(ready)}, Still running: {len(not_ready)}")

# Cancel remaining tasks
for ref in not_ready:
    ray.cancel(ref)

<div class="alert alert-info">
Refer to the <strong><a href="https://docs.ray.io/en/latest/ray-core/tasks/retries.html" target="_blank">retries</a></strong> and <strong><a href="https://docs.ray.io/en/latest/ray-core/tasks/fault-tolerance.html" target="_blank">fault tolerance</a></strong> documentation to learn more.
</div>

## 7. Task runtime environments

A runtime environment defines dependencies such as files, packages, and environment variables needed for a Python script to run.

- **Runtime Environment Management**:
  - Managed by a `RuntimeEnvAgent` gRPC server on each node.
  - The `RuntimeEnvAgent` fate-shares with the raylet, simplifying the failure model and ensuring it is a core component for task and actor scheduling.

- **Environment Creation**:
  - Triggered by the raylet via a gRPC request to the `RuntimeEnvAgent` when a task or actor requires a runtime environment.
  - May involve:
    - Installing packages using `pip install`.
    - Setting environment variables for Ray worker processes.
    - Activating conda environments with `conda activate`.
    - Downloading files from remote cloud storage.

- **Resource Caching**:
  - Runtime environment resources, such as downloaded files and installed conda environments, are cached on each node.
  - The cache allows sharing of resources between different tasks, actors, and jobs.
  - When the cache size limit is exceeded, resources not currently in use are deleted to free up space.

Here is a diagram showcasing the above concepts:

<img src="https://anyscale-materials.s3.us-west-2.amazonaws.com/ray-core/runtime_env.png" width="500">

### 7.1. Setting environment variables

For example, we can set an environment variable:


In [ ]:
@ray.remote(runtime_env={"env_vars": {"my_custom_env": "prod"}})
def f():
    env = os.environ["my_custom_env"]
    return f"My custom env is {env}"

In [ ]:
ray.get(f.remote())

### 7.2. Installing pip dependencies


In [ ]:
@ray.remote(runtime_env={"pip": ["requests", "pandas==1.5.0"]})
def fetch_data(url):
    import requests
    return requests.get(url).json()

### 7.3. Working directory


In [ ]:
@ray.remote(runtime_env={"working_dir": "s3://my-bucket/project/my_directory.zip"})
def load_config():
    with open("config.yaml") as f:
        return f.read()

### 7.4. Note about pip dependencies

pip dependencies in task runtime environments don't come for free. They add to the startup time of the worker process.

**Performance impact:**
- First task: 10-60 seconds (installing dependencies)
- Subsequent tasks: < 1 second (cached environment)

**Best practices:**
- If you need the same dependencies across many tasks, bake them into your cluster image
- Use conda environments for better caching
- Group tasks with same dependencies together

If you find yourself needing to install the same dependencies across many tasks, consider baking them into the image you use to start your Ray cluster.

## 8. Resource allocation and management

### 8.1. Understanding logical vs physical resources

Ray resource specifications are **logical**, not physical:

- **Logical resources**: Used by Ray scheduler for placement decisions (default: `num_cpus=1`)
- **Physical resources**: Actual CPU/GPU/memory usage by your code

In [ ]:
@ray.remote(num_cpus=1)  # Ray reserves 1 CPU slot for scheduling
def cpu_intensive_task():
    # Ray sets OMP_NUM_THREADS=1 to match num_cpus
    return np.dot(large_matrix_a, large_matrix_b)

**Key points:**
- `num_cpus` is a scheduling hint, not a hard limit
- Ray automatically sets `OMP_NUM_THREADS` to match `num_cpus` to prevent oversubscription

You can override this if needed (may cause oversubscription)

In [ ]:
@ray.remote(num_cpus=1)
def mm(n: int = 4000):
    return np.dot(np.random.rand(n, n), np.random.rand(n, n))

# Override to use 8 threads (caution: may oversubscribe)
ray.get(mm.options(runtime_env={"env_vars": {"OMP_NUM_THREADS": "8"}}).remote())

Note assigning "GPU" resources to a task, Ray will automatically set the `CUDA_VISIBLE_DEVICES` env var within the worker to limit it to specific GPU ids.

<div class="alert alert-info">
Learn more about <strong><a href="https://docs.ray.io/en/latest/ray-core/scheduling/resources.html#physical-resources-and-logical-resources" target="_blank">physical resources and logical resources</a></strong>.
</div>

### 8.2. Fractional resources for I/O-bound tasks

Ray supports **fractional CPU requests** (e.g., `num_cpus=0.1`, `num_cpus=0.5`) to enable efficient oversubscription of I/O-bound tasks.

**When to use fractional CPUs:**

Tasks that spend most of their time waiting (not computing) can share CPU slots:

In [ ]:
# Moderately I/O-bound: Some computation, some I/O
@ray.remote(num_cpus=0.5)  # Allow 2 tasks per CPU core
def download_and_parse(url):
    data = requests.get(url).text
    return parse_large_file(data)

**Benefits:**
- **Higher throughput**: Run more tasks concurrently when they're waiting on I/O
- **Better resource utilization**: Don't waste CPU cores on tasks that are mostly idle
- **Cost efficiency**: Process more work on the same hardware

<div class="alert alert-warning">

**Note:** Don't abuse fractional resources and fall into the anti-pattern of launching too many small tasks. Instead, batch work and leverage multi-threading within tasks when possible.

</div>

In [ ]:
# GPU model serving: Load 4 small models on one GPU
@ray.remote(num_gpus=0.25)
def run_small_model(model, input_data):
    return model.predict(input_data)

<div class="alert alert-info">
Fractional resources include support for <strong><a href="https://docs.ray.io/en/latest/ray-core/scheduling/accelerators.html#fractional-accelerators" target="_blank">multiple accelerators</a></strong>, allowing users to load multiple smaller models onto a single GPU. Learn more about <strong><a href="https://docs.ray.io/en/latest/ray-core/scheduling/resources.html#fractional-resource-requirements" target="_blank">fractional resource requirements</a></strong>.
</div>

### 8.3. Checking available resources

<p>During the <em>scheduling stage</em>, Ray evaluates the <strong>resource requirements</strong> specified via the <code>@ray.remote</code> decorator or within the <code>resources={...}</code> argument. These requirements may include:</p>

<ul>
    <li><strong>CPU</strong> e.g., <code>@ray.remote(num_cpus=2)</code>)</li>
    <li><strong>GPU</strong> e.g., <code>@ray.remote(num_gpus=1)</code>)</li>
    <li><strong>Custom resources</strong>: User-defined custom resources like <code>"TPU"</code></li>
    <li><strong>Memory</strong></li>
</ul>

<p>Ray's scheduler checks the <strong>resource specification</strong> (sometimes referred to as <strong>resource shape</strong>) to match tasks and actors with available resources in the cluster. If the exact resource combination is unavailable, Ray may autoscale the cluster.</p>

<p>You can inspect the current resource availability using:</p>


In [ ]:
# Total cluster resources
ray.cluster_resources()

In [ ]:
# Currently available (not reserved)
ray.available_resources()

<div class="alert alert-info">

<strong>Pattern:</strong> configure the head node to be unavailable for compute tasks.

When scaling to large clusters, it's important to ensure that the <strong>head node</strong> does not handle any compute tasks. Users can indicate that the head node is unavailable for compute by setting its resources:

```resources: {"CPU": 0}```

Learn more about <strong><a href="https://docs.ray.io/en/latest/cluster/vms/user-guides/large-cluster-best-practices.html#configuring-the-head-node" target="_blank">configuring the head node</a></strong>.
</div>

## 9. Pipeline data processing and waiting for results

After launching a number of tasks, you may want to know which ones have finished executing without blocking on all of them. This could be achieved by `ray.wait()`

### 9.1. Understanding ray.wait()

`ray.wait()` is a powerful primitive for building pipelines and managing task completion:

**Signature:**

In [ ]:
@ray.remote
def remote_fn(x):
    time.sleep(random.uniform(2, 10))
    return x

refs = [remote_fn.remote(i) for i in range(10)]

ready, not_ready = ray.wait(
    refs,
    num_returns=1,      # Number of references to wait for
    timeout=None,       # Max time to wait for (seconds)
    fetch_local=True    # Whether to fetch objects to the local node or not
)

In [ ]:
ready

**Returns:**
- `ready`: List of ObjectRefs that are ready
- `not_ready`: List of ObjectRefs still pending

### 9.2. Pipeline pattern with ray.wait()


| <img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/ray-core/pipeline-data-processing.png" width="400px" loading="lazy">                                                                               |
| :----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| (top panel) Execution timeline when using ray.get() to wait for all results before calling process results. (bottom panel) Execution timeline when using ray.wait() to process results as soon as they become available. |

Here are functions to match the above diagram:

In [ ]:
@ray.remote
def do_some_work(x):
    time.sleep(x)
    return x

@ray.remote
def process_incremental(result):
    time.sleep(1)
    return result * 2

@ray.remote
def process_results(results):
    out = []
    for result in results:
        time.sleep(1)
        out.append(result * 2)
    return out

This is the **naive approach:** block until all tasks are complete and then process the results.


In [ ]:
inputs = [2, 3, 1, 4]
start = time.time()
data_list = ray.get([do_some_work.remote(x) for x in inputs])
output = ray.get(process_results.remote(data_list))
print("duration =", time.time() - start, "\nresult = ", output)
# Duration: ~8 seconds (4s max task + 4s processing)

This is the **pipelined** approach: process items as soon as they become available


In [ ]:
start = time.time()
result_ids = [do_some_work.remote(x) for x in inputs]
refs = []
while len(result_ids):
    done_id, result_ids = ray.wait(result_ids, num_returns=1)
    print(done_id)
    refs.append(process_incremental.remote(done_id))
output = ray.get(refs)
print("duration =", time.time() - start, "\nresult = ", sum)
# Duration: ~5 seconds (overlapping ~4s computation and 1s processing)

<div class="alert alert-info">
Read more about the <strong><a href="https://docs.ray.io/en/latest/ray-core/tips-for-first-time.html#tip-4-pipeline-data-processing" target="_blank">pipeline data processing</a></strong>
</div>

## 10. Ray generators

[Ray Generators](https://docs.ray.io/en/latest/ray-core/ray-generator.html) are a way to make use of the python generator pattern to generate data.

They are useful for:
- Reducing worker heap memory usage **by** avoiding building up a large in-memory collection
- Reducing object store memory usage **by** allowing for garbage collection of objects that are processed

### 10.1. Why use Ray Generators?

**Problem with regular tasks:**


In [ ]:
@ray.remote
def produce_large_dataset():
    # Creates all data in memory at once
    results = []
    for i in range(100):
        results.append(np.random.rand(1000, 1000))  # Each object is ~8MB
    return results  # ~800MB in memory!

# High memory pressure
ref = produce_large_dataset.remote()

**Solution with generators:**


In [ ]:
@ray.remote
def produce_large_dataset():
    # Yields one object at a time
    for i in range(100):
        yield np.random.rand(1000, 1000)  # Only ~8MB at a time

# Process streaming
for obj_ref in produce_large_dataset.remote():
    result = ray.get(obj_ref)
    process(result)
    # Previous objects can be garbage collected

### 10.2. Python generator recap

Let's start with a sample python generator function:


In [ ]:
def generator_function():
    for i in range(10):
        time.sleep(1)
        yield i

Here is how we can iterate over the generator function:


In [ ]:
for obj in generator_function():
    print(obj)

### 10.3. Converting to Ray generator

Converting into a Ray generator function is straightforward:


In [ ]:
@ray.remote
def generator_function():
    for i in range(10):
        time.sleep(1)
        yield i

In [ ]:
for obj_ref in generator_function.remote():
    print(obj_ref)  # Prints ObjectRef

result = ray.get(obj_ref)
print(result)  # Prints actual value

### 10.4. Memory usage comparison

See the below script which shows the memory consumption when running with and without a generator:


In [ ]:
!RAY_DEDUP_LOGS=0 python scripts/ray_generator_object_store_diff.py

### 10.5. Key differences from Python generators

Unlike python generators, Ray generators:
- **Don't pause execution** - i.e. they don't require `__next__` to be called to yield the next element
- **Don't support all APIs** like send and throw
- **Execute eagerly** - the task runs to completion regardless of consumer speed

Given that Ray eagerly executes a generator task to completion regardless of whether the caller is polling the partial results or not, it might lead to object store spilling.

### 10.6. Backpressure control

To backpressure the generator, we can specify the `_generator_backpressure_num_objects` argument in the `@ray.remote` decorator:


In [ ]:
@ray.remote(num_returns="streaming", _generator_backpressure_num_objects=10)
def generate_data():
    for i in range(1000):
        yield expensive_square(i)
        # Will pause if consumer is slow and 10 objects are buffered

Below is a script that shows execution with and without backpressure:


In [ ]:
!python scripts/ray_generator_backpressure_diff.py

### 10.7. When to use Ray Generators

**✅ Use Ray Generators when:**
- Processing large datasets that don't fit in memory
- Streaming results from long-running computations
- Building data pipelines with multiple stages
- You want incremental results (don't wait for everything)

**❌ Don't use Ray Generators when:**
- You need random access to results
- You need the full result set at once